# Titanic data

Dane mozna pobrać po utworzeniu (darmowego) konta na portalu Kaggle. 

[Pobierz dane](https://www.kaggle.com/c/titanic/data):
interesują nas tylko pliki zbiorów `train.csv` i `test.csv`.

Zobaczmy jak wyglądają nasze dane:

In [ ]:
import pandas as pd

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

print("train ma {} wierszy i {} kolumn".format(*train.shape))
print("test ma {} wierszy i {} kolumn".format(*test.shape))

print(f"train to obiekt typu {type(train)}")

In [ ]:
train.info()

Metoda `info()` zwraca informacje o:
- nazyach kolumn, 
- ich indeksy,
- liczbę niepustych (`null`) elementów dla kazdej kolumny,  
- typy danych.

In [ ]:
test.info()

Dla zbioru testowego mamy jedną kolumnę (`Survived`) mniej, dlaczego?  

Ze względu, iz nie planujemy wrzucać wyników modeli na kaggle zbiór test nie jest nam potrzebny.

Informacje z metody `info()` przedstawiają tylko ogólne rzeczy, zobaczmy jak zbiór train wygląda w środku.

In [ ]:
train.head()

Kazda kolumna reprezentuje jedną zmienną naszych danych. Identyfikatorem, bądź kluczem naszej tabeli jest `PassengerId`, która przyjmuje rózną wartość
dla kazdego wiersza. Czy taka zmienna moze być dobra do modelowania? 
Zmienna `Survived` realizuje zmienną celu naszego zadania - pasazer przezyl (1) lub nie (0). 
`Pclass` to zmienna opisująca klasę pokładu zgodnie z biletem.

### Czyszczenie danych

Nasze dane zawierają zarówno dane numeryczne jak i kategoryczne. Niektóre kategorie reprezentowane są przez wartości liczbowe, a niektóre przez tekst.

Na podstawie metody `info()` wiemy równiez, ze nie wszystkie kolumny mają zmienne wypełnione całkowicie. 

Większość algorytmów ML nie radzi sobie z brakami danych. Istnieją trzy podstawowe opcje jak mozemy sobie z tym poradzić:
1. usunięcie wierszy w których pojawiają się jakieś braki danych.
2. usunięcie całej kolumny gdzie występują braki danych
3. Wypełnienie brakujących wartości (imputacja danych) zerem, wartością średnią, lub medianą.

In [ ]:
# opcja 1 - tylko 2 pasazerow nie maja Embarked - nie znamy portu docelowego - mozemy usunac te wiersze
train = train.dropna(subset=['Embarked'])

# opcja 2 - tutaj mamy tylko 204 wiersze z wartosciami w kolumnie Cabin - mozemy usunac te kolumne
train = train.drop("Cabin", axis=1)

# opcja 3 - znamy wiek 714 pasazerow. Dlatego opcja 2 nie jest dobra. Opcja 1 tez nie jest dobra bo usuniemy $22\%$ danych.
mean = train['Age'].mean()
train['Age'] = train['Age'].fillna(mean)

train.info()

print('Zmienna PassengerId ma {} roznych wartosci'.format(train['PassengerId'].nunique()))

In [ ]:
print('Zmienna PassengerId ma {} roznych wartosci'.format(train['PassengerId'].nunique()))
print('Zmienna Name ma {} roznych wartosci'.format(train['Name'].nunique()))
print('Zmienna Ticket ma {} roznych wartosci'.format(train['Ticket'].nunique()))

In [ ]:
train = train.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

train.info()

Zmienne tekstowe

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in ['Sex','Embarked']:
    le.fit(train[col])
    train[col] = le.transform(train[col])

train.head()

Skalowanie danych

In [ ]:
print('max wieku to {}'.format(train['Age'].max())) 
print('max zmiennej Fare to {}'.format(train['Fare'].max()))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train[['Age', 'Fare']])
train[['Age', 'Fare']] = scaler.transform(train[['Age', 'Fare']])
train.head()

In [ ]:
# test - uwaga na zwracany typ danych
sc = MinMaxScaler()
sc.fit(train)
tr=sc.transform(train)
print(type(tr),tr)

In [ ]:
#### Podział na zbiór treningowy i testowy
from sklearn.model_selection import train_test_split

input_data = train.iloc[:, 1:8]
labels = train.iloc[:,0]

tr_input, test_input, tr_labels, test_labels = train_test_split(input_data, labels, test_size=0.2, random_state=42)

In [ ]:
tr_input.shape, test_input.shape

In [ ]:
import numpy as np 
with open('../data/train.npy', 'wb') as f:
    np.save(f, tr_input)
    np.save(f, tr_labels)

with open('../data/test.npy', 'wb') as f:
    np.save(f, test_input)
    np.save(f, test_labels)

## Klasyfikatory

In [87]:
import random
random.seed(42)

# losowa funkcja klasyfikująca
def classify(passanger):
    return random.randint(0,1)

# pomocnicza funkcja
def run(f_classufy, x):
    return list(map(f_classufy, x))

def evaluate(predictions, actual):
    correct = list(filter(
        lambda item: item[0] == item[1],
        list(zip(predictions, actual))
    ))
    return f"{len(correct)} poprawnych przewidywan z {len(actual)}. Accuracy ({len(correct)/len(actual)*100:.0f}%)"

In [ ]:
evaluate(run(classify, tr_input.values), tr_labels.values)


In [ ]:
def kill_bill(item):
    return 0

In [ ]:
evaluate(run(kill_bill, tr_input.values), tr_labels.values)

In [ ]:
from sklearn.metrics import confusion_matrix

predictions = run(kill_bill, tr_input.values)
confusion_matrix(tr_labels.values, predictions)
# TN, FP, FN, TP

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(tr_labels.values, predictions))
print(recall_score(tr_labels.values, predictions))
print(f1_score(tr_labels.values, predictions))

# specificity = \sum TrueNegatives / \sum ALLActualNegatives
# npv = \sum TrueNegatives / \sum AllPredictedNegatives

def specificity(matrix):
    return matrix[0,0]/(matrix[0][0]+matrix[0][1]) if (matrix[0][0]+matrix[0][1] > 0) else 0

def npv(matrix):
    return matrix[0,0]/(matrix[0][0]+matrix[1][0]) if (matrix[0][0]+matrix[1][0] > 0) else 0

cm = confusion_matrix(tr_labels.values, predictions)
print("specificity",specificity(cm))
print("npv",npv(cm))

> Zrob obliczenia dla losowego klasyfikatora!

In [88]:
def raport(name, run, classify, input, labels):
    cr_predictions = run(classify, input.values)
    cr_cm = confusion_matrix(labels.values, cr_predictions)
    cr_prcision  = precision_score(labels.values, cr_predictions)
    cr_recall = recall_score(labels.values, cr_predictions)
    cr_scpecificity = specificity(cr_cm)
    cr_npv = npv(cr_cm)
    cr_level = 0.25*(cr_prcision + cr_recall + cr_scpecificity + cr_npv)
    print(f"{name} precision {cr_prcision:.2f} recall {cr_recall:.2f} specificity {cr_scpecificity:.2f} npv {cr_npv:.2f} level {cr_level:.2f}")
    

In [89]:
raport("losowy", run, classify, tr_input, tr_labels)
raport("kill bill", run, kill_bill, tr_input, tr_labels)

losowy precision 0.37 recall 0.47 specificity 0.50 npv 0.61 level 0.49
kill bill precision 0.00 recall 0.00 specificity 1.00 npv 0.62 level 0.40


/Users/air/Desktop/quarto_projects/intro_to_qml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from math import sqrt

qc = QuantumCircuit(1)
initial_state = [1/sqrt(2), 1/sqrt(2)]
qc.initialize(initial_state, 0)
qc.measure_all()

Powyzszy obwód realizuje stan superpozycji i zwraca w losowy sposób wynik $0$ lub $1$.
Oznacza to, ze moze byc kandydatem na klasyfikator binarny.

Zdefiniujmy powyzszy obwód tak aby realizowany był jako funkcja, którą mozemy wykorzystać w naszym problemie klasyfikacji. 

In [90]:
from qiskit import execute, Aer, QuantumCircuit
from math import sqrt 
from sklearn.metrics import confusion_matrix, precision_score, recall_score 

def pqc_classify(backend, passenger_state):
    qc = QuantumCircuit(1)
    qc.initialize(passenger_state, 0)
    qc.measure_all()
    result = execute(qc, backend, shots=1).result()
    counts = result.get_counts()
    return int(list(map(lambda item: item[0], counts.items()))[0])

backend = Aer.get_backend('qasm_simulator')
initial_state = [1/sqrt(2), 1/sqrt(2)]

In [91]:
raport("Random PQC", run, lambda x: pqc_classify(backend, initial_state), 
       tr_input, tr_labels)

Random PQC precision 0.40 recall 0.53 specificity 0.50 npv 0.64 level 0.52


Powyzsze kody realizują klasyfikatory, które nie zalezą od naszych danych pasazerów. 

1. Preprocessing - przetworznie danych wejściowych do postaci przetwarzanej przez nasz obwód kwantowy. Wykorzystamy PQC. 
Ta część jest związana z klasycznym przetworzeniem danych i utworzeniem embeddingu. 
2. PQC
3. Postprocessing - Nasz klasyfikator powinien zwracać wartość 0 lub 1. Tutaj powinien odbywać się proces przetłumaczenia wyniku realizowanego 
przez jakiś obwód kwanotwy na binarny wynik klasyfikacji. Tutaj równiez uzyjemy PQC do klasycznego przetworzenia.

Tylko druga część będzie w pełni realizowała obwód kwantowy. Łącząc wszystko razem otrzymujemy `Wariacyjny hybrydowy klasyczno-kwantowy algorytm`.
Jest to jedno z najczęściej uzywanych podejść do modelowania danych klasycznych. 

In [92]:
# 1 preprocessing
def pre_process(passanger):
    quantum_state = [1/sqrt(2), 1/sqrt(2)]
    return quantum_state

# 2. pqc

def pqc(beckend, quantum_state):
    qc = QuantumCircuit(1)
    qc.initialize(quantum_state, 0)
    qc.measure_all()
    result = execute(qc, backend, shots=1).result()
    counts = result.get_counts(qc)
    return counts

# 3. postprocessing
def post_process(counts):
    return int(list(map(lambda item: item[0], counts.items()))[0])

backend = Aer.get_backend('qasm_simulator')

raport("Variational Classifier", run, lambda passenger: post_process(pqc(backend, pre_process(passenger))), 
       tr_input, tr_labels)

Variational Classifier precision 0.41 recall 0.55 specificity 0.50 npv 0.65 level 0.53


Dane kazdego pasazera składają się z 7 zmiennych. Ze względu, iz nie chcemy (na razie) zmieniać ostatniego kroku musimy znaleźć jakąś metodę 
pozwalającą przypisać 7 zmiennym prawdopodobieństwo przezycia i śmierci. W ostatnim kroku odczytujemy po pomiarze tylko te dwie wielkości. 

Znalezienie prawdopodobieństwa dla 7 zmiennych nie jest prostym zadaniem (w końcu to robią nasze klasyczne modele ML). Jendak mozemy zacząć 
od bardzo statystycznego podejścia. Zakładamy, ze zmienne są od siebie niezalezne i kazda zmienna z jakąś wagą przyczynia się do wartości prawdopodobieństwa przezycia. 
$$
P(survival) = \sum (F \mu_F)
$$ 


In [93]:
def waga_zmiennej(feature, weight):
    return feature*weight

from functools import reduce

def get_overall_probablity(features, weights):
    return reduce(lambda result, data: result + waga_zmiennej(*data), 
                  zip(features, weights),
                  0
                  )

Jak zbudować wektor wag? 

Zacznijmy od `współczynnika korelacji`.



In [94]:
from scipy.stats import spearmanr

columns = [list(map(lambda passneger: passneger[i], tr_input.values)) for i in range(0,7)]

In [95]:
correlations = list(map(lambda col: spearmanr(col, tr_labels.values)[0], columns))

In [96]:
correlations

[-0.33362848376406934,
 -0.5327583106581802,
 -0.03158046336028065,
 0.0688875885695018,
 0.12641683959850614,
 0.3105976636091728,
 -0.16652847475942076]

Zastosujmy to do pre-processingu

In [97]:
from math import pi, sin, cos 

def get_state(theta):
    return [cos(theta/2), sin(theta/2)]

def pre_process_weighted(passenger):
    mu = get_overall_probablity(passenger, correlations)
    # theta między 0 i pi  0 = |0> a pi = |1>
    quantum_state = get_state((1-mu)*pi)
    return quantum_state

In [100]:
backend = Aer.get_backend('statevector_simulator')

raport("Variational Classifier - train", run, lambda passenger: post_process(pqc(backend, pre_process_weighted(passenger))), 
       tr_input, tr_labels)
raport("Variational Classifier- test", run, lambda passenger: post_process(pqc(backend, pre_process_weighted(passenger))), 
       test_input, test_labels)

Variational Classifier - train precision 0.28 recall 0.37 specificity 0.42 npv 0.52 level 0.39
Variational Classifier- test precision 0.31 recall 0.38 specificity 0.48 npv 0.55 level 0.43
